#  TAYSIR Baseline for Track 1- Extraction of DFA from Recurrent Neural Net already Trained on a Binary Classification Task

### Welcome!

This is a notebook to let you play around with the Deterministic Finite Automata (DFA) extraction baseline that use the k-means clustering algorithm on the hidden states of the model encountered while parsing the validation set.

Inputed Neural Net can be LSTM, GRU or SRN network, after which it will draw a neat little DFA.

# Requirements
## Imports and version verifying 

In [ ]:
%pip install -q mlflow torch

In [ ]:
import torch
import mlflow

print("Your torch version:", torch.__version__)
print("Your mlflow version:", mlflow.__version__)
import sys
print("Your python version:", sys.version)


This notebook was tested with:
* Torch version: 1.11.0+cu102
* MLFlow version: 1.25.1
* Python version: 3.8.10 [GCC 9.4.0]

Python versions starting at 3.7 are supposed to work (but have not been tested).

## Choosing the task

First you must select one of the phases/datasets we provide

In [ ]:
TRACK = 1
DATASET = 0

## Loading the RNN of the competition


In [ ]:
model_name = f"models/{TRACK}.{DATASET}.taysir.model"

model = mlflow.pytorch.load_model(model_name)
model.eval()

### Initialisation of some variables that would be useful

In [ ]:
nb_letters = model.input_size -1
cell_type = model.cell_type

print("The alphabet contains", nb_letters, "symbols.")
print("The type of the recurrent cells is", cell_type.__name__)

## Load the data

The input data is in the following format :

```
[Number of sequences] [Alphabet size]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
...
[Length of sequence] [List of symbols]
```

For example the following data :

```
5 10
6 8 6 5 1 6 7 4 9
12 8 6 9 4 6 8 2 1 0 6 5 9
7 8 9 4 3 0 4 9
4 8 0 4 9
8 8 1 5 2 6 0 5 3 9
```

is composed of 5 sequences and have an alphabet size of 10 (so symbols are between 0 and 9) and the first sequence is composed of 6 symbols (8 6 5 1 6 7 4 9), notice that 8 is the start symbol and 9 is the end symbol.

In [ ]:
file = f"datasets/{TRACK}.{DATASET}.taysir.valid.words"

sequences = []
with open(file) as f:
    f.readline() #Skip first line (number of sequences, alphabet size)
    for line in f:
        line = line.strip()
        seq = line.split(' ')
        seq = [int(i) for i in seq[1:]] #Remove first value (length of sequence) and cast to int
        sequences.append(seq)

The variable *sequences* is thus **a list of lists**.

In [ ]:
print('Number of sequences:', len(sequences))
print('10 first sequences:')
for i in range(10):
    print(sequences[i])

We then load the trained RNN. It is given as a MLFlow model.

# Model extraction
## Seeding
We are seeding for reproductibility:

In [ ]:
import random
import numpy as np
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

## Get the latent states
We first define a function that computes and stores the latent states reached by the RNN during the processing of the available data. 

Notice that it is possible to batch this function and to use GPU, but we provide a basic version so that it would be more clear to understand how our RNN can be used.

In [ ]:
def get_latent(model, sequences):
    """
    This function parses the dataset into the model and keeps track of the hidden state of the model it saw during the process
    returns the observed hidden states
    input: the MLFlow model, the dataset
    output: observed hidden states in a matrix
    """
    with torch.no_grad():  
        h_lists = []
        
        for w in sequences:
            
            one_hot_w = model.one_hot_encode(w)
            
            """Get tensors of hidden states encountered at each layer"""
            h_tensors = model.reached_hidden(one_hot_w, None)
            """
            h_tensor is a list of list of states. The number of lists is the number of words (here one)
                Each list is made of a number of tensors corresponding to the number of hidden layer
                    Each tensor contains the hidden states encountered at this layer while parsing the word. 
                    Dimension:(1, w_length, size of the layer) 
            """ 
            h_tensors = h_tensors[0] # because we just passed one word
            if model.task == 'lm':
                h_tensors = h_tensors[-1] # to use the full string, not a prefix 
            
            """We need to go through the layers to get the states of the model, that is, the vectors concatenating the states of each layers"""
            hiddens = []
            for j in range(model.n_layers):
                if model.hides_pairs: #True if LSTM, False otherwise 
                    (h,c) = h_tensors[j]
                    #h contains the vectors outputed by the layer at each timestep/length, c contains the corresponding carries
                    
                    """Get vectors concatening h and c for all prefix lengths"""
                    #the squeeze got rid of the batch size dimension (here valued 1), the unbind allows to have one vector per prefix length
                    l = list(zip(h.squeeze(0).unbind(0),
                                c.squeeze(0).unbind(0)))
                    #l contains the list of (h, c) encountered at this layer
                    l= [torch.cat(x) for x in l]
                    #Now l contains vectors corresponding to the concatenation of h and c at each prefix length

                else:
                    #non LSTM
                    h = h_tensors[j]
                    l = h.squeeze(0).unbind(0)
                    
                hiddens.append(l)
                    
            """Merge the different lists: the state of a RNN is the concatenantion of the states of its recurrent layers"""
            ls = list(map(list, zip(*hiddens))) # transpose
            ls = [torch.cat(x) for x in ls]
            #Make the vectors of hidden states into a matrix:
            ls = torch.stack(ls)
            
            h_lists += [ls]
            # So h_lists contains tensors, each one containing the hidden states encountered for a given word in sequences. Dim: (w_length, hidden_size)
            
    return torch.cat(h_lists)

In [ ]:
H = get_latent(model, sequences)

## DFA Extraction Baseline
This part is the one you need to change to put your own algorithm.


The general idea of this algorithm is to parse words into the RNN and store the hidden states then run the k-means algorithm
on these vectors to get the state of the DFA. To find the transition, we use the centroid of
each cluster as starting point for the RNN and look in which cluster we arrive for each letter. The information
about the final states is obtained by feeding the centroid vector to the output layer(s), and initial state is an
initial distribution over the abstract states.

<ol>
<li> <b>Collecting hidden state vectors</b> to get space <i>H</i> (Lines 1-6). At the first part,  we retrieve all hidden state values from RNN on the set of input sequences <i>X</i> and store them as <i>H</i> (Line 6). Moreover, we take out the hidden state values from all layers and directions, in the case of LSTM also retrieve cell state values.</li>
<li><b>Fitting k-means model</b> on the latent values <i>H</i> to get the clusters <i>Q</i>  of k-means model (Lines 8-9)</li>
<li><p> <b>Creating DFA set:</b> transition matrices E_&sigma;, final <i>F</i> and initial <i>I</i> states  (Lines 11-25):</p>
<ul>
<li>  <b> Transition matrices E_&sigma; (Lines 11-26)</b>. Firstly, we initialize zero matrices and vectors (Lines 11-15) and after to fill transition matrices we use two loops (Lines 16-17) that help to go through all amount of clusters and all elements of vocabulary. The next step is getting
    the output features <i>out</i> from the last layer of the RNN  and <i>h</i> that contains the final hidden state (and cell state in the case of LSTM)
    from all layers and directions (Line 18).
     After getting an id of centroid <i>q'</i>  by classifying every <i>h</i> value with k-means model, we use <i>q'</i> to fill E_&sigma; with 1's (Lines 19-20).</li>
<li><b> Final state vector <i>F</i> (Lines 21-23)</b>. To get the final state we use <i>out</i> on dense layer of trained RNN (Line 21) and if   <i>sigmoid(dense_out)</i> greater than or equal 0.5 fill <i>F[q]</i> with 1, otherwise it remains zero (Lines 22-23). 
<li> <b>Initial state vector <i>I</i> (Lines 24-26)</b>. To get id of element what be filled with 1, we classify <i>h_0</i> by k-means model and fill <i>I</i>. After when everything is done, we are able to create DFA set that contains <i>(I, F, E_&sigma;)</i>



In [ ]:
from IPython.display import Image
Image("algo_DFA.png")

### Baseline Algorithm
And now we code our baseline Algorithm

We use the data structure for Automaton that can be found in the toolbox *scikit-splearn* (more info at https://remieyraud.github.io/scikit-splearn/).

The toolbox can easily be installed using:

In [ ]:
!pip3 install scikit-splearn

In [ ]:
from sklearn.cluster import KMeans
from splearn.automaton import Automaton

def extract_DFA(model, latent_values, n_clusters, nb_letters, random_state = 0):

    """ 
    Parameters
    ----------
    model : object
        RNN model
    latent_values : stack of hidden states observed
    n_clusters : int
        Amount of clusters for K-means algorithm
    nb_letters : list
        List of the symbols in the sequences of the dataset

    Returns
    -------
    DFA : object
        Returns a DFA        
    """
    
    kmeans = KMeans(init="k-means++", n_clusters=n_clusters, n_init=5, random_state = random_state)
    kmeans.fit(latent_values.cpu())


    Q = kmeans.cluster_centers_ # line 9 / centroids representing 
    I = torch.zeros(n_clusters) # line 11
    F = torch.zeros(n_clusters) # line 11
    trans_list = []  #line 12 - initialisation of the list of transition matrices

    for i in range(nb_letters):   # line 13
        trans_matrix = np.zeros((n_clusters, n_clusters)) # line 14
        trans_list.append(trans_matrix) # line 15
    
    with torch.no_grad():
        D = 2 if model.bidirectional else 1
        LSTM = model.hides_pairs #True if each layer has hidden state formated as (h,c), that is, is made of LSTM cells
        
        for q in range(n_clusters): #line 16 
            #Get the hidden state q corresponding to the centroid under consideration
            centroid = torch.Tensor(Q[q])
            
            """Format the centroid into the model latent dimension to use it as a starting state in the model"""
            if LSTM: #
                #Get values for each layers
                state = centroid.view(model.n_layers,-1)
                #get the hidden state of each layer separated from the carry
                (h, c) = torch.tensor_split(state, 2, dim=-1)
                h = h.unbind(0)
                #Take care of the potential bidirectional behavior
                h = [x.view(D, -1) for x in h]
                
                #same for the carry
                c = c.unbind(0)
                c = [x.view(D, -1) for x in c]
                
                h_centroid = list(zip(h,c))
            else:
                #Get values for each layers
                state = centroid.view(model.n_layers,-1)
                h = state.unbind(0)
                #Take care of the potential bidirectional behavior
                h_centroid = [x.view(D, -1) for x in h]
                
            for letter in range(nb_letters): # line 17               
                """Get the hidden state reached from the centroid using the current letter"""
                
                #First we encode the letter
                one_hot_l = model.one_hot_encode([letter])
                
                #Then we forward this letter inside the RNN from the hidden state corresponding to the centroid    
                if LSTM: 
                    output, h_c_list = model(one_hot_l, h_centroid)
                    #h_c_list is the state reached. We format it in a vector following the same way that get_latent()
                    v = [torch.cat(tuple(map(torch.flatten, x)), dim=-1) for x in h_c_list]
                    h_reach = torch.cat(v)
                else:
                    output, h_list = model(one_hot_l, h_centroid)
                    #h_list is the state reached from centroid using current letter. We format it in the same way that get_latent()
                    v = [torch.flatten(x) for x in h_list]
                    h_reach = torch.cat(v)
                                                
                #transform hidden state obtained to k_means format
                h_kmeans = h_reach
                                                            
                #get reached cluster
                q_prime = kmeans.predict(h_kmeans.reshape(1, -1)) #lines 19 - 20
                q_prime = q_prime[0]
                                                
                #Modify correspond Automata transition matrix
                trans_list[letter][q][q_prime] = 1 #line 21
            
            """Is the state accepting? That is, what is the output of the dense layr(s) from the centroid""" 
            #Format of dense is different from h_centroid because only the last hidden layer is fed to dense one(s) - without c for LSTM
            h = state[-1,:model.neurons_per_layer*D]
            out = model.feed_dense(h)
                                                 
            F[q] = out.argmax()
            
        """Who is the initial state?"""
        #Initial state is always 0's, but the dimensions vary with the architecture. Easiest way to have it in the needed k-means format: 
        h_0 = torch.zeros_like(centroid)                                    
        q_0 = kmeans.predict(h_0.reshape(1, -1))[0]
        I[q_0]=1

    DFA = Automaton(nbL=nb_letters, nbS=n_clusters, initial = np.array(I, dtype = float), final=np.array(F, dtype = float),   # line 28
                transitions=trans_list, type='classic')

    return DFA

## Running the DFA extraction baseline

In [ ]:
#Construction the DFA
DFA = extract_DFA(model, H, n_clusters = 12, nb_letters = nb_letters)
print("Number of states of the extracted DFA:", DFA.initial.shape[0])
print("Output on example:", DFA.val(sequences[42]))

# Submission
Save extracted DFA as a MLFlow Model. This is the creation of the model needed for the submission to the competition. 

The only thing to do is to define a function that takes a sequence as a list of integers and returns the value given to this sequence to the sequence. Your model is **NOT** a parameter of this function.  

In [ ]:
def predict(seq):
    return 0 if DFA.val(seq) < 0.5 else 1

## Save and submit 
This is the creation of the model needed for the submission to the competition: you just have to run this cell. It will create in your current directory an **archive**  that you can then submit on the competition website.

**You should NOT modify this part, just run it**

In [ ]:
from submit_tools import save_function

save_function(predict, alphabet_size=nb_letters, prefix=f'dataset{TRACK}.{DATASET}')

# For fun, show DFA graphical representation
You may need to install the graphviz library.

In [ ]:
dot =DFA.get_dot(threshold = 0.01, title = 'dotfile')
# To display the dot string, one can use graphviz:
from graphviz import Source
src = Source(dot)
src.render('dotfile' + '.gv', view=True)